In [1]:
# dependencies
import pandas as pd
import os

In [2]:
# set file paths
schoolsPath = os.path.join("Resources", "schools_complete.csv")
studentsPath = os.path.join("Resources", "students_complete.csv")

# read in csv files
schools = pd.read_csv(schoolsPath)
students = pd.read_csv(studentsPath)

In [3]:
# combine both files into one csv dataset
schoolsNstudents = pd.merge(students, schools, on="school_name", how="left")
#schoolsNstudents

In [4]:
# calculate the total number of schools
unique_schools = schoolsNstudents["school_name"].unique()
total_schools = len(unique_schools)
total_schools

15

In [5]:
# calculate the total number of students
total_students = len(schoolsNstudents["student_name"])
total_students

39170

In [6]:
# calculate the total budget
total_budget = schools["budget"].sum()
total_budget

24649428

In [7]:
# calculate the average math score
average_math = students["math_score"].mean()
average_math

78.98537145774827

In [8]:
# calculate the average reading score
average_reading = students["reading_score"].mean()
average_reading

81.87784018381414

In [9]:
# calculate the percentage of students with a passing math score (70 or greater)
math_greater_70 = students.loc[students["math_score"] >= 70, :]
math_passing_percent = (len(math_greater_70) / total_students)*100
math_passing_percent

74.9808526933878

In [10]:
# calculate the percentage of students with a passing reading score (70 or greater)
reading_greater_70 = students.loc[students["reading_score"] >= 70, :]
reading_passing_percent = (len(reading_greater_70) / total_students)*100
reading_passing_percent

85.80546336482001

In [11]:
# calculate the percentage of students who passed math and reading (% Overall Passing)
passing_math_reading = students.loc[(students["math_score"] >= 70) & (students["reading_score"] >= 70), :]
overall_passing_percent = (len(passing_math_reading) / total_students)*100
overall_passing_percent

65.17232575950983

In [12]:
# Create a dataframe to hold the above results
district_summary = pd.DataFrame({"Total Schools": [total_schools],
                        "Total Students": [total_students],
                        "Total Budget": [total_budget],
                        "Average Math Score": [average_math],
                        "Average Reading Score": [average_reading],
                        "% Passing Math": [math_passing_percent],
                        "% Passing Reading": [reading_passing_percent],
                        "% Overall Passing": [overall_passing_percent],
                        })

In [13]:
# converting integers to float for formatting
district_summary["Total Students"] = district_summary["Total Students"].astype(float)
district_summary["Total Budget"] = district_summary["Total Budget"].astype(float)

In [14]:
# formatting for better read of district summary
district_summary["Total Students"] = district_summary["Total Students"].map('{:,.0f}'.format)
district_summary["Total Budget"] = district_summary["Total Budget"].map('${:,.2f}'.format)
district_summary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,65.172326


In [15]:
# calculate budget per student by schools
schools["Per Student Budget"] = schools["budget"] / schools["size"]
schools = schools.sort_values("school_name", ascending=True)
#sschools

In [16]:
math_score_per_school = students.set_index("school_name").groupby("school_name") ["math_score"].sum()
students_per_school = students.set_index("school_name").groupby("school_name")["math_score"].count()
average_math_score_df = math_score_per_school / students_per_school
#average_math_score_df

In [17]:
reading_score_per_school = students.set_index("school_name").groupby("school_name") ["reading_score"].sum()
average_reading_score_df = reading_score_per_school / students_per_school
#average_reading_score_df

In [18]:
# calculate the percentage of students with a passing math score (70 or greater) per school
math_greater_70_per_school = students.loc[students["math_score"] >= 70, :]
group_math_greater_70_per_school = math_greater_70_per_school.groupby("school_name") ["math_score"].count()
math_passing_pecent_per_school_df = (group_math_greater_70_per_school / students_per_school) * 100
#math_passing_pecent_per_school_df

In [19]:
# calculate the percentage of students with a passing reading score (70 or greater) per school
reading_greater_70_per_school = students.loc[students["reading_score"] >= 70, :]
group_reading_greater_70_per_school = reading_greater_70_per_school.set_index("school_name").groupby("school_name") ["reading_score"].count()
reading_passing_pecent_per_school_df = (group_reading_greater_70_per_school / students_per_school) * 100
#reading_passing_pecent_per_school_df

In [20]:
# calculate the percentage of students who passed math and reading (% Overall Passing) per school
passing_math_reading_per_school = students.loc[(students["math_score"] >= 70) & (students["reading_score"] >= 70), :]
group_passing_math_reading_per_school = passing_math_reading_per_school.groupby("school_name") ["school_name"].count()
overall_passing_percent_per_school_df = (group_passing_math_reading_per_school / students_per_school)*100
#overall_passing_percent_per_school_df

In [21]:
# Collected all require student marks in a summary dataframe
student_marks_summary_df = pd.DataFrame({"Average Math Score": average_math_score_df,
                                         "Average Reading Score": average_reading_score_df,
                                         "% Passing Math": math_passing_pecent_per_school_df,
                                         "% Passing Reading": reading_passing_pecent_per_school_df,
                                         "% Overall Passing": overall_passing_percent_per_school_df
                                          })
#student_marks_summary_df

In [60]:
# merge the schools and the student marks summary for school summary output; along with viewable formats
school_summary = pd.merge(schools, student_marks_summary_df, on="school_name", how="left")
del school_summary["School ID"]
school_summary = school_summary.rename(columns={"school_name": "School Name", 
                                                "type": "School Type",
                                                "size": "Total Students",
                                                "budget": "Total School Budget"                                    
                                                })
school_summary["Total Students"] = school_summary["Total Students"].map('{:,.0f}'.format)
school_summary["Total School Budget"] = school_summary["Total School Budget"].map('${:,.2f}'.format)
school_summary["Per Student Budget"] = school_summary["Per Student Budget"].map('${:.2f}'.format)
#school_summary = school_summary.style.set_properties(**{"text-align": "left"}).hide_index()
school_summary = school_summary.style.set_properties(**{'text-align': 'left'}).set_table_styles([ dict(selector='th', props=[('text-align', 'left')] ) ])
school_summary

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,Bailey High School,District,"4,976","$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.642283
1,Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
2,Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
3,Ford High School,District,"2,739","$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,54.289887
4,Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
5,Hernandez High School,District,"4,635","$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
6,Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,89.227166
7,Huang High School,District,"2,917","$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
8,Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172
9,Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541
